In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# Ruta del archivo CSV
file_path = '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/datamunging/consolidated_data.csv'

# Leer el archivo CSV en un DataFrame
df = pd.read_csv(file_path)

# Verificar el tamaño del DataFrame
print(f"Tamaño del DataFrame: {df.shape}")

# Establecer valores NaN para ciertas columnas
puntos_interes = ['cinema', 'restaurant', 'theatre', 'police', 'supermarket',
                   'pharmacy', 'bar', 'playground', 'cafe', 'monument', 'station',
                   'clinic', 'post_office', 'convenience', 'sports_centre', 'atm',
                   'memorial', 'museum', 'artwork', 'bicycle_rental', 'library', 
                   'waste_disposal', 'platform', 'viewpoint', 'school', 'fuel', 'works', 
                   'bus_station', 'clothes', 'fire_station', 'college', 'hospital', 
                   'doctors', 'mall', 'stop_position', 'park', 'prison', 'fast_food', 
                   'garden', 'marketplace', 'fountain', 'townhall', 'industrial', 
                   'ferry_terminal', 'public_building', 'place_of_worship', 'bench', 
                   'drinking_water', 'gym', 'stop_area', 'bird_hide', 'food_court', 'shelter', 
                   'information', 'stationery', 'farm', 'yoga', 'attraction', 'mobile_library', 
                   'chalet', 'parking_entrance', 'parking', 'wholesale', 'bakery', 'pub', 
                   'vending_machine', 'pastry', 'yes', 'sailing_school', 'windsurfing_rental', 
                   'ice_cream', 'internet_cafe', 'clock', 'dentist']

df.loc[:, puntos_interes] = np.nan

# Crear la nueva columna 'precio/mt2'
df['precio/mt2'] = np.where(df['mt2'] != 0, df['precio'] / df['mt2'], np.nan)

# Eliminar las columnas 'precio' y 'mt2'
df = df.drop(columns=['precio', 'mt2'])

# Eliminar filas donde 'precio/mt2' es NaN, 0, o infinito
df = df[~df['precio/mt2'].isin([0, np.inf, -np.inf]) & df['precio/mt2'].notna()]

# Separar en DataFrames de alquiler y venta
df_alquiler = df[df['alquiler_venta'] == 'alquiler'].drop(columns=['alquiler_venta'])
df_venta = df[df['alquiler_venta'] == 'venta'].drop(columns=['alquiler_venta'])

# Función para realizar regresión
def realizar_regresion(df):
    # Separar características y variable objetivo
    X = df.drop('precio/mt2', axis=1)
    y = df['precio/mt2']

    # Imputar valores NaN en y con la mediana
    y = y.fillna(y.median())

    # Identificar columnas numéricas
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns

    # Preprocesamiento para columnas numéricas
    num_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('scaler', StandardScaler())
    ])

    # Combinar preprocesadores solo para columnas numéricas
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_cols)
        ])

    # Crear un pipeline para la regresión con RandomForest
    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X[num_cols], y, test_size=0.2, random_state=42)

    # Verificar tamaños de conjuntos de entrenamiento y prueba
    print(f"Tamaño de X_train: {X_train.shape}")
    print(f"Tamaño de X_test: {X_test.shape}")

    # Entrenar el modelo de RandomForest
    rf_pipeline.fit(X_train, y_train)

    # Realizar predicciones
    y_pred_rf = rf_pipeline.predict(X_test)

    # Evaluar el modelo
    mae_rf = mean_absolute_error(y_test, y_pred_rf)
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    rmse_rf = np.sqrt(mse_rf)
    r2_rf = r2_score(y_test, y_pred_rf)

    print("Resultados de RandomForest:")
    print(f'MAE: {mae_rf}')
    print(f'RMSE: {rmse_rf}')
    print(f'R2 Score: {r2_rf}')

    # Imprimir algunas predicciones para verificar
    print("\nAlgunas predicciones de RandomForest:")
    for i in range(5):
        print(f"Precio real: {y_test.iloc[i]}, Precio predicho: {y_pred_rf[i]}")

    # Obtener importancia de características
    rf_model = rf_pipeline.named_steps['regressor']
    importances = rf_model.feature_importances_
    feature_names = rf_pipeline.named_steps['preprocessor'].get_feature_names_out()

    # Crear un DataFrame con la importancia de características
    importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
    importance_df = importance_df.sort_values('importance', ascending=False)

    print("\nCaracterísticas más importantes de RandomForest:")
    print(importance_df.head(10))

    print("RandomForest completada.")

# Realizar regresión para df_alquiler
print("Regresión para Alquiler:")
realizar_regresion(df_alquiler)

# Realizar regresión para df_venta
print("\nRegresión para Venta:")
realizar_regresion(df_venta)

/var/folders/gk/kfm0w04x18d1tr94k7ml70yc0000gn/T/ipykernel_4439/3827076252.py:15: DtypeWarning: Columns (11,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Tamaño del DataFrame: (175700, 18)
Regresión para Alquiler:
Tamaño de X_train: (43417, 80)
Tamaño de X_test: (10855, 80)
Resultados de RandomForest:
MAE: 7.132816553024214
RMSE: 13.108114055617662
R2 Score: 0.2999378995951736

Algunas predicciones de RandomForest:
Precio real: 22.857142857142858, Precio predicho: 19.145338764355905
Precio real: 12.604166666666666, Precio predicho: 13.78256350163246
Precio real: 37.432432432432435, Precio predicho: 35.21651829809271
Precio real: 39.0234375, Precio predicho: 27.59790502240076
Precio real: 38.983050847457626, Precio predicho: 38.1865006987783

Características más importantes de RandomForest:
                feature  importance
2         num__latitude    0.255499
5   num__publicado_hace    0.229679
3        num__longitude    0.184068
0     num__habitaciones    0.162883
1           num__banios    0.133636
4           num__planta    0.034234
52           num__bench    0.000000
59     num__information    0.000000
58         num__shelter    0.